# Tenacity - Israel Zúñiga de la Mora

## Python Monterrey: Primera reunión 2019 
### 22/Febrero/2019 07:00 P.M. @ WeWork (Blvd. Antonio L. Rodriguez 1888)

![Pymty](py-mty-logo.png)

# `whois(me)`

Personal:

* 89
* Colima, Col. 🇲🇽

Uni|Grado|Años
------|---------|----
Universidad de Colima|Ing. en Telemática|2008-2012
Universidad Abierta y a Distancia de México (UnADM)|Lic. en Matemáticas|2018-∞
Collective Academy|Master of Business & Technology|2019-2020
Universidad de la Calle|Perreología|?
----

Profesional:
* Latest: Chief Customer Intelligence Officer @ Evolve Fintech / Ictineo PTF
* UDC >> TecLID A.C. ✝ >> Intel Corp. >> ZapopanLab >> Invictus.mx >> Alturin ✝ >> Banregio
----
* https://github.com/israelzuniga  >>> https://github.com/israelzuniga/slides  (para encontrar esta presentación)  
* https://www.linkedin.com/in/israelzuniga/
* https://twitter.com/0xD1

# Colabora y participa con la comunidad!

## Framewoks, bibliotecas de código, módulos, patrones, challenges, pypy, cpython, threading, Uc, etc

## Meetup Local: https://www.facebook.com/groups/pymty/  | https://www.meetup.com/pythonmty/
## Otras comunidades cercanas: PythonDayMX, PyLATAM 2019 (P.V.), PyTexas (Austin/abril), SciPy (Austin/junio)

# Designing for Failure

> Exception handling is one of the most brushed aside aspects of computer programming. Errors are complicated to handle, and often they are unlikely, so developers always forget to handle failures... sometimes they even forgot on purpose.

> However, in a world where applications are distributed over the network, across miles of fiber optic cable and on different computers, failure is not an exception. It must be considered as the norm for your software. Failure scenarios must be first-citizens of the various testing scenarios being developed.

> In an environment distributed over a network, anything that can fail **will** fail.

> Python does not offer any help in that regard, and almost no programming language offers advanced error recovery or retryng capability -- except mabe languages implementing [condition systems](https://en.wikipedia.org/wiki/Exception_handling#Condition_systems) such as Common Lisp.

-- Julien Danjou - The Hacker's Guide to Scaling Python (2017)

# Retry pattern / Retrying pattern


Habilita una aplicación para conducir fallas transitorias cuando intenta conectarse a un servicio o recurso de red, volviendo a intentar de forma transparente la operación fallida. Pudiendo mejorar la estabilidad de la aplicación.





[MS Azure Architecture/Cloud Design Patterns/Retry pattern](https://docs.microsoft.com/en-us/azure/architecture/patterns/retry)



## Contexto y Problema
Una aplicación que se comunica con elementos corriendo en la nube, debe ser sensitiva a fallas transitorias que pueden ocurrir en este entorno. Las fallas incluyen la pérdida momentánea de conectividad de la red a los componentes y servicios, la indisponibilidad temporal de un servicio o los tiempos de espera que se producen cuando un servicio está ocupado.

Estas fallas suelen ser autocorregibles, y si la acción que desencadenó una falla se repite después de un retraso adecuado, es probable que tenga éxito. 

## Soluciones
* Cancelar
* Volver a procesar (retry)
* Volver a procesar después de un retraso (retry after delay)

[MS Azure Architecture/Cloud Design Patterns/Retry pattern](https://docs.microsoft.com/en-us/azure/architecture/patterns/retry)

![](https://docs.microsoft.com/en-us/azure/architecture/patterns/_images/retry-pattern.png)



[MS Azure Architecture/Cloud Design Patterns/Retry pattern](https://docs.microsoft.com/en-us/azure/architecture/patterns/retry)

# Problemas y consideraciones

* Ajustar la política de reintentos según los requisitios de la aplicación. Para algunas operaciones no críticas, es mejor fallar rápido en lugar de volver a intentarlo varias veces e impactar el rendimiento de la aplicación.

* Una política agresiva de reintento con un retraso mínimo entre intentos y un gran número de reintentos podría degradar aún más un servicio ocupado que se está ejecutando cerca o en su capacidad. 
Debe considerar los siguientes puntos al decidir cómo implementar este patrón.

* Si la lógica de negocio lo permite, optar por  reportar o guardar la excepción y pasar a un recurso más degradado. Esto para mejorar la estabilidad y resilencia de la aplicación. Según el patrón [Circuit Breaker](https://docs.microsoft.com/en-us/azure/architecture/patterns/circuit-breaker)


[MS Azure Architecture/Cloud Design Patterns/Retry pattern](https://docs.microsoft.com/en-us/azure/architecture/patterns/retry)

# Cuando NO usar este patrón

* Cuando es probable que una falla sea duradera: La aplicación podría estar desperdiciando tiempo y recursos tratando de repetir una solicitud que es probable que falle.
   
* Para el manejo de fallas que no se deben a fallas transitorias, como las excepciones internas causadas por errores en la lógica de negocios de una aplicación.
 
* Como alternativa a la solución de problemas de escalabilidad en un sistema. Si una aplicación experimenta fallos frecuentes y ocupados, a menudo es una señal de que el servicio o recurso al que se accede debe ampliarse.



[MS Azure Architecture/Cloud Design Patterns/Retry pattern](https://docs.microsoft.com/en-us/azure/architecture/patterns/retry)

# Cuando  SI usar este patrón

* Cuando una aplicación podría experimentar fallas transitorias cuando interactúa con un servicio remoto o accede a un recurso remoto.

* Cuando los fallos sean de corta duración, y la repetición de una solicitud que previamente haya fallado podría tener éxito en un intento posterior.

[MS Azure Architecture/Cloud Design Patterns/Retry pattern](https://docs.microsoft.com/en-us/azure/architecture/patterns/retry)

# ¿Por que te debería importar?

Añadir un manejo inteligente de excepciones permite a tu aplicación/codigo falle correctamente.

# Casos de uso
* Web scraping
* Conexiones a servidores remotos (p. ej. ssh+paramiko, DBs, confirmaciones de transferencia de información)
* Carga/Descarga de archivos
* Código que pueda fallarCode that can fail


In [95]:
import time
import random
from datetime import datetime

def do_something():
    cur_time = datetime.now().strftime('%H:%M:%S:%f')
    if random.randint(0,1) == 0:
        print('{0}: Failure'.format(cur_time))
        raise RuntimeError
    print('{0}: Success'.format(cur_time))

# Naive Retrying

## Retrying pattern


In [55]:
while True:
    try:
        do_something()
    except:
        pass
    else:
        break

22:37:42:028459: Failure
22:37:42:032881: Failure
22:37:42:032997: Failure
22:37:42:033089: Success


## Retrying pattern w/sleep
Espera un número fijo de segundos antes de volver a procesar.


In [57]:
 while True:
    try:
        do_something()
    except:
        # Nos dormimos mamalon por 5 seg antes de retry
        time.sleep(5)
    else:
        break
        

22:37:47:271282: Failure
22:37:52:272306: Success


## Retrying pattern w/exponential backoff
Espera con base en un retroceso exponencial (1 seg  -> 2 -> 4 -> 8 -> 16 -> etc)

In [60]:
attempt = 0
while True:
    try:
        do_something()
    except:
        # Dormimos 2^attemp seg antes de reintentar
        time.sleep(2 ** attempt)
        attempt += 1
    else:
        break

22:38:26:508438: Failure
22:38:27:511175: Failure
22:38:29:515638: Failure
22:38:33:519442: Failure
22:38:41:522556: Success


# Meet: Tenacity
Tenacity es una biblioteca para Python 2 y 3, diseñada para reintentar la ejecución de una tarea o proceso cuando ocurre una `Exception`. Es un fork de [retry](https://github.com/rholder/retrying). El mantenedor es [Julien Danjou](https://julien.danjou.info/).

Repo: https://github.com/jd/tenacity

# Features
* [Funcionalidad de `Decorator`](https://www.thecodeship.com/patterns/guide-to-python-function-decorators/)
* Especificación de condiciones:
    * tiempo en espera/wait (fijo, exponencial, aleatorio)
    * cancelación (límite por tiempo, o número de intentos)
* Configurable por excepción
* Retry para co-rutinas [(`asyncio`)](https://docs.python.org/3/library/asyncio-task.html)

# Instalación y uso

Para instalar desde la consola:

```bash
pip install tenacity
```

Para usar:

In [96]:
import tenacity

# Wait Conditions

# Retrying  w/ fixed sleep
Espera un número fijo de segundos antes de volver a procesar.


In [92]:


@tenacity.retry(wait=tenacity.wait_fixed(1))
def el_veinte():
    '''
    funct: el_veinte
    Input: ()
    Output: Exception si randint != 1, True si randint == 1.
    Description:
    Simula un dado de 20 caras.
    Un dado de 20 caras, representa un icosaedro. 
    Si las veinte caras del icosaedro son triángulos equiláteros y congruentes, 
    iguales entre sí, el icosaedro es convexo y se denomina regular, 
    siendo entonces uno de los llamados sólidos platónicos.
    '''
    random_num = random.randint(1,20)
    cur_time = datetime.now().strftime('%H:%M:%S:%f')
    if random_num != 20:
        print('{0}: {1} no es igual a 20'.format(cur_time, random_num))
        raise Exception()
    print('{0}: 20 Fue seleccionado !!!!11!!'.format(cur_time))
    

el_veinte()



16:25:50:658249: 6 no es igual a 20
16:25:51:660911: 2 no es igual a 20
16:25:52:664249: 11 no es igual a 20
16:25:53:669708: 15 no es igual a 20
16:25:54:672792: 4 no es igual a 20
16:25:55:673536: 8 no es igual a 20
16:25:56:674168: 5 no es igual a 20
16:25:57:679594: 18 no es igual a 20
16:25:58:682999: 19 no es igual a 20
16:25:59:688120: 1 no es igual a 20
16:26:00:693673: 2 no es igual a 20
16:26:01:696516: 11 no es igual a 20
16:26:02:698842: 6 no es igual a 20
16:26:03:703908: 19 no es igual a 20
16:26:04:706002: 16 no es igual a 20
16:26:05:708800: 14 no es igual a 20
16:26:06:712063: 18 no es igual a 20
16:26:07:713329: 3 no es igual a 20
16:26:08:717932: 2 no es igual a 20
16:26:09:718989: 18 no es igual a 20
16:26:10:722880: 10 no es igual a 20
16:26:11:727333: 17 no es igual a 20
16:26:12:728488: 15 no es igual a 20
16:26:13:730023: 6 no es igual a 20
16:26:14:734189: 17 no es igual a 20
16:26:15:734880: 9 no es igual a 20
16:26:16:739927: 1 no es igual a 20
16:26:17:74179

# Retry w/random backoff
Espera con base en un número aleatorio de segundos

In [93]:
@tenacity.retry(wait=tenacity.wait_random(5, 10))
def el_veinte():
    '''
    funct: el_veinte
    Input: ()
    Output: Exception si randint != 1, True si randint == 1.
    Description:
    Simula un dado de 20 caras.
    Un dado de 20 caras, representa un icosaedro. 
    Si las veinte caras del icosaedro son triángulos equiláteros y congruentes, 
    iguales entre sí, el icosaedro es convexo y se denomina regular, 
    siendo entonces uno de los llamados sólidos platónicos.
    '''
    random_num = random.randint(1,20)
    cur_time = datetime.now().strftime('%H:%M:%S:%f')
    if random_num != 20:
        print('{0}: {1} no es igual a 20'.format(cur_time, random_num))
        raise Exception()
    print('{0}: 20 Fue seleccionado !!!!11!!'.format(cur_time))
    

el_veinte()


16:26:52:594659: 3 no es igual a 20
16:27:00:294516: 16 no es igual a 20
16:27:07:383964: 2 no es igual a 20
16:27:16:451135: 6 no es igual a 20
16:27:24:781455: 6 no es igual a 20
16:27:32:402761: 2 no es igual a 20
16:27:42:261575: 15 no es igual a 20
16:27:47:507201: 6 no es igual a 20
16:27:56:441516: 3 no es igual a 20
16:28:04:480305: 8 no es igual a 20
16:28:09:674600: 7 no es igual a 20
16:28:18:927768: 8 no es igual a 20
16:28:25:505337: 16 no es igual a 20
16:28:33:347031: 10 no es igual a 20
16:28:38:835794: 9 no es igual a 20
16:28:46:333062: 18 no es igual a 20
16:28:54:303894: 3 no es igual a 20
16:29:00:802118: 5 no es igual a 20
16:29:08:008318: 4 no es igual a 20
16:29:14:985664: 18 no es igual a 20
16:29:24:563565: 4 no es igual a 20
16:29:30:902571: 3 no es igual a 20
16:29:40:542498: 8 no es igual a 20
16:29:48:214349: 3 no es igual a 20
16:29:55:311268: 3 no es igual a 20
16:30:01:885645: 3 no es igual a 20
16:30:06:907654: 20 Fue seleccionado !!!!11!!


## Tenacity: Retry w/exponential backoff
Espera con base en un retroceso exponencial (1 seg  -> 2 -> 4 -> 8 -> 16 -> etc)

In [94]:
@tenacity.retry(wait=tenacity.wait_exponential())
def el_veinte():
    '''
    funct: el_veinte
    Input: ()
    Output: Exception si randint != 1, True si randint == 1.
    Description:
    Simula un dado de 20 caras.
    Un dado de 20 caras, representa un icosaedro. 
    Si las veinte caras del icosaedro son triángulos equiláteros y congruentes, 
    iguales entre sí, el icosaedro es convexo y se denomina regular, 
    siendo entonces uno de los llamados sólidos platónicos.
    '''
    random_num = random.randint(1,20)
    cur_time = datetime.now().strftime('%H:%M:%S:%f')
    if random_num != 20:
        print('{0}: {1} no es igual a 20'.format(cur_time, random_num))
        raise Exception()
    print('{0}: 20 Fue seleccionado !!!!11!!'.format(cur_time))
    

el_veinte()

16:30:06:921101: 6 no es igual a 20
16:30:08:926372: 18 no es igual a 20
16:30:12:929463: 11 no es igual a 20
16:30:20:932434: 1 no es igual a 20
16:30:36:936868: 17 no es igual a 20
16:31:08:937723: 12 no es igual a 20
16:32:12:938469: 6 no es igual a 20
16:34:20:939974: 15 no es igual a 20
16:38:36:939481: 16 no es igual a 20
16:47:08:931023: 10 no es igual a 20
17:04:12:911741: 17 no es igual a 20


KeyboardInterrupt: 

# Stop Conditions

# Fixed Number of Retries

Cantidad de intentos: Vuelve a procesar cada dos segundos hasta que se hayan cumplido cinco intentos

In [97]:
@tenacity.retry(wait=tenacity.wait_fixed(2),
                stop=tenacity.stop_after_attempt(5))
def wtshtf():
    print(datetime.now().strftime('%H:%M:%S:%f'))
    raise Exception()
try:    
    wtshtf()
except Exception:
    print('EXCEPTION!')

17:11:53:819141
17:11:55:822428
17:11:57:827719
17:11:59:828338
17:12:01:828759
EXCEPTION!


# Time

Intentos sobre plazo fijo en el tiempo: Vuelve a procesar cada 0.5 segundos hasta que hayan pasado 3 segundos

In [98]:
@tenacity.retry(wait=tenacity.wait_fixed(0.5),
                stop=tenacity.stop_after_delay(3))
def wtshtf():
    print(datetime.now().strftime('%H:%M:%S:%f'))
    raise Exception()
try:    
    wtshtf()
except Exception:
    print('EXCEPTION!')

17:12:30:388368
17:12:30:894619
17:12:31:399929
17:12:31:905250
17:12:32:409840
17:12:32:915231
17:12:33:419149
EXCEPTION!


#  Retry for Exception Type

Vuelve a procesar cada n tiempo, pero solo si obtenemos una excepción: Intenta cada 0.5 segundos, pero solo para la excepción de tipo `IOError`.


In [112]:
@tenacity.retry(retry=tenacity.retry_if_exception_type(IOError),
                wait=tenacity.wait_fixed(0.5))
def corrupcion_wave():
    random_num = random.randint(0, 9)
    cur_time = datetime.now().strftime('%H:%M:%S:%f')
    if random_num > 2:
        print('{0} - {1}.txt no existe.'.format(cur_time, random_num))
        raise IOError()
    elif random_num == 2:
        print('{0} - {1}.txt está corrupto.'.format(cur_time, random_num))
        raise SystemError()
    print('{0} - 1.txt fue seleccionado!!!1'.format(cur_time))

try:
    corrupcion_wave()
except Exception:
    print('EXCEPTION! 😝')

17:15:50:924521 - 4.txt no existe.
17:15:51:426214 - 9.txt no existe.
17:15:51:927214 - 9.txt no existe.
17:15:52:429181 - 5.txt no existe.
17:15:52:934023 - 4.txt no existe.
17:15:53:439671 - 5.txt no existe.
17:15:53:943557 - 8.txt no existe.
17:15:54:446276 - 7.txt no existe.
17:15:54:946747 - 9.txt no existe.
17:15:55:450525 - 5.txt no existe.
17:15:55:953993 - 6.txt no existe.
17:15:56:458988 - 7.txt no existe.
17:15:56:963827 - 5.txt no existe.
17:15:57:468018 - 6.txt no existe.
17:15:57:972030 - 2.txt está corrupto.
EXCEPTION!


In [117]:
@tenacity.retry(retry=tenacity.retry_if_exception_type(IOError),
                wait=tenacity.wait_fixed(0.5))
def corrupcion_wave():
    random_num = random.randint(0, 9)
    cur_time = datetime.now().strftime('%H:%M:%S:%f')
    if random_num > 2:
        print('{0} - {1}.txt no existe.'.format(cur_time, random_num))
        raise IOError()
    elif random_num == 2:
        print('{0} - {1}.txt está corrupto.'.format(cur_time, random_num))
        raise SystemError()
    print('{0} - 1.txt fue seleccionado!!!1'.format(cur_time))

try:
    corrupcion_wave()
except Exception:
    print('EXCEPTION! 😝')

17:16:42:177650 - 2.txt está corrupto.
EXCEPTION! 😝


# ¿Por que te debería importar? (II)

![](https://d2908q01vomqb2.cloudfront.net/fc074d501302eb2b93e2554793fcaf50b3bf7291/2017/10/03/exponential-backoff-and-jitter-blog-figure-7.png)

* [AWS Architecture Blog: Exponential Backoff And Jitter](https://aws.amazon.com/blogs/architecture/exponential-backoff-and-jitter/)
* [AWS Architecture Blog: Jitter & Backoff simulator](https://github.com/aws-samples/aws-arch-backoff-simulator)

(R: Spike control)

# ¿Por que te debería importar? (II)

![](https://d2908q01vomqb2.cloudfront.net/fc074d501302eb2b93e2554793fcaf50b3bf7291/2017/10/03/exponential-backoff-and-jitter-blog-figure-12.png)

[AWS Architecture Blog: Exponential Backoff And Jitter](https://aws.amazon.com/blogs/architecture/exponential-backoff-and-jitter/)
[AWS Architecture Blog: Jitter & Backoff simulator](https://github.com/aws-samples/aws-arch-backoff-simulator)

-------

* [AWS Architecture Blog: Exponential Backoff And Jitter](https://aws.amazon.com/blogs/architecture/exponential-backoff-and-jitter/)
* [AWS Architecture Blog: Jitter & Backoff simulator](https://github.com/aws-samples/aws-arch-backoff-simulator)